# Spherical samplings for weather prediction

In [ ]:
# Cartopy uses deprecated mpl features (MatplotlibDeprecationWarning).
import warnings
from matplotlib.cbook import MatplotlibDeprecationWarning
warnings.simplefilter("ignore", MatplotlibDeprecationWarning)

import numpy as np
import scipy.spatial
from scipy.spatial.distance import pdist, squareform
from scipy.spatial import SphericalVoronoi, geometric_slerp
import pygsp as pg
import healpy as hp
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import proj3d
import cartopy.crs as ccrs

In [ ]:
def plot_spherical_graph(graph):
    print(graph)
    fig = plt.figure(figsize=(17, 5))

    ax = fig.add_subplot(1, 3, 1, projection='3d')
    graph.set_coordinates('sphere', dim=3)
    graph.plot(indices=True, ax=ax, title='3D')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')

    ax = fig.add_subplot(1, 3, 2)
    graph.set_coordinates('sphere', dim=2)
    graph.plot(indices=True, ax=ax, title='Equirectangular projection')
    ax.set_xlabel('longitude [radians]')
    ax.set_ylabel('latitude [radians]')

    # ax = fig.add_subplot(1, 3, 3, projection='mollweide')
    # graph.plot(indices=True, ax=ax, title='Mollweide projection (equi-area)')

    crs = ccrs.Mollweide()
    ax = fig.add_subplot(1, 3, 3, projection=crs)
    ax.coastlines()
    ax.set_global()
    graph.set_coordinates('sphere', dim=2)
    graph.coords *= 180/np.pi
    graph.coords = crs.transform_points(ccrs.Geodetic(), *graph.coords.T)[:, :2]
    graph.plot(indices=True, ax=ax, title='Mollweide projection')

    fig.tight_layout()

## 1 Spherical grids

The sphere graphs are created with the [PyGSP](https://pygsp.readthedocs.io).
If the data is created from another tool, it is important to make sure the vertices/cells are ordered in the same way.
An easy way to check is to plot the data on the sampled sphere with `graph.plot(myfield)`.

### 1.1 Equiangular (Driscoll-Healy, Clenshaw-Curtis)

* Resolution parameter: number of isolatitude rings $N_{lat}$ (and pixels per ring, often $N_{lon} = 2 N_{lat}$).
* Number of pixels: $N_{pix} = N_{lat} \times N_{lon}$.
* Number of pixels around the equator: $N_{lon}$.

In [ ]:
# Illustration.
graph = pg.graphs.SphereEquiangular(4, 8, poles=0)
#graph = pg.graphs.SphereEquiangular(4, 8, poles=2)
plot_spherical_graph(graph)

### 1.2 (Reduced) Gauss-Legendre

ECMWF: octahedral reduced Gaussian grid, named `O320` for $N=320$.

* Resolution parameter: number of isolatitude rings $N_{lat} = 2N$.
* Number of pixels: $4N(N+9)$.
* Number of pixels around the equator: $4N+16$.

References:
* <https://confluence.ecmwf.int/display/FCST/Introducing+the+octahedral+reduced+Gaussian+grid>
* <https://confluence.ecmwf.int/display/OIFS/4.2+OpenIFS%3A+Octahedral+grid>

In [ ]:
# Illustration.
graph = pg.graphs.SphereGaussLegendre(6, nlon='ecmwf-octahedral', k=10)
#graph = pg.graphs.SphereGaussLegendre(6, k=8)
plot_spherical_graph(graph)

### 1.3 HEALPix

* Resolution parameter: number of subdivisions $L$ ($N_{side}$).
* Number of pixels: $12 L^2$.
* Number of pixels around the equator: $4 L$.

In [ ]:
# Illustration.
graph = pg.graphs.SphereHealpix(2, k=8)
#graph = pg.graphs.SphereHealpix(2, k=8, nest=True)
plot_spherical_graph(graph)

In [ ]:
# Compare with healpy (geographical vs astrophysical flip).
hp.mollview(graph.signals['lon'], flip='geo')

In [ ]:
# Percentage of the sphere attainable by a filter.
# The number of neighbors k is proportional to area.
kernel_size = 3
G = pg.graphs.SphereHealpix(16, k=40)
(G.L**(kernel_size-1)).nnz / G.N**2 *100

### 1.4 Icosahedral

* Resolution parameter: number of subdivisions $L$.
* Number of pixels: $10 L^2 + 2$ (vertices, hexagonal cells, `dual=False`) or $20 L^2$ (faces, triangular cells, `dual=True`).
* Number of pixels around the equator: $\approx 4L$ or $5L$.
    * The subdivided icosahedron has no prefered orientation, nor isolatitude rings.

In [ ]:
# Illustration.
graph = pg.graphs.SphereIcosahedral(2, dual=False)
#graph = pg.graphs.SphereIcosahedral(2, dual=True)
plot_spherical_graph(graph)

In [ ]:
# Distances between pixels become less and less constant as resolution increases.
graph = pg.graphs.SphereIcosahedral(8, dual=True, k=3)
dist = squareform(pdist(graph.coords))
dist *= graph.A.toarray()
dist = dist.flatten()
dist = dist[dist!=0]
plt.hist(dist, bins=100);

### 1.5 Cubed-sphere

Used by the [US Global Forecasting Model](https://www.gfdl.noaa.gov/fv3/fv3-grids/).

* Resolution parameter: number of subdivisions $L$.
* Number of pixels: $6L^2$.
* Number of pixels around the equator: $4L$.

In [ ]:
graph = pg.graphs.SphereCubed(3, 'equiangular')
#graph = pg.graphs.SphereCubed(3, 'equidistant')
plot_spherical_graph(graph)

## 2 Resolutions

Comparison:
1. Same average resolution (area, sqrt(area)=angle) <=> same number of pixels.
2. Same average resolution near equator (equatorial band) -> different for non-uniform samplings.

Comments:
* All pixels in HEALPix have the same area. The Icosahedral and reduced Gauss-Legendre are mostly equiarea. The Equiangular is not.

Procedure:
1. Choose the number of subdivisions for HEALPix and Icosahedral (as they are the least flexible ones).
2. Compute the resulting number of pixels (averaged between the two).
3. Choose parameters for Equiangular and Gauss-Legendre to approach that target number of pixels.
4. Add another Equiangular with 50% more pixels. It will have about the same resolution as the others at the equator.

In [ ]:
def params2npix(sampling, params):
    if sampling == 'equiangular':
        nlat, nlon = params
        return nlat*nlon
    elif sampling == 'gaussian':
        nlat = params
        assert (nlat % 2) == 0
        nlat //= 2
        return 4 * nlat * (nlat+9)
    elif sampling == 'healpix':
        subdivisions = params
        return 12 * subdivisions**2
    elif sampling == 'icosahedral':
        subdivisions = params
        return 10 * subdivisions**2 + 2
    elif sampling == 'cubed':
        subdivisions = params
        return 6 * subdivisions**2

def npix2params(sampling, npix):
    if sampling == 'equiangular':
        nlat = round(np.sqrt(npix/2))
        return nlat, 2*nlat
    elif sampling == 'gaussian':
        a, b, c = 4, 36, -npix
        sol = (-b + np.sqrt(b**2 - 4*a*c)) / (2*a)
        nlat = 2*sol
        return nlat
    elif sampling == 'healpix':
        subdivisions = np.sqrt(npix / 12)
        return subdivisions
    elif sampling == 'icosahedral':
        subdivisions = np.sqrt((npix-2) / 10)
        return subdivisions
    elif sampling == 'cubed':
        subdivisions = np.sqrt(npix / 6)
        return subdivisions

assert npix2params('equiangular', params2npix('equiangular', (100, 200))) == (100, 200)
assert npix2params('gaussian', params2npix('gaussian', 80)) == 80
assert npix2params('healpix', params2npix('healpix', 5)) == 5
assert npix2params('icosahedral', params2npix('icosahedral', 5)) == 5
assert npix2params('cubed', params2npix('cubed', 8)) == 8

In [ ]:
def npix2res(npix, height=1):
    radius = 6371  # radius of the Earth
    height = 2 * height * radius
    return np.sqrt(2*np.pi*radius*height/npix)

def plot_resolutions(graphs):
    # TODO: smooth with non-square window, e.g., a Gaussian.
    avg = np.pi/180*12.3456789
    bins = np.linspace(avg/2, np.pi/2-avg/2, 100)
    hist = np.empty_like(bins)
    fig, ax = plt.subplots(figsize=(10, 8))
    
    for graph in graphs:
        lat = abs(graph.signals['lat'])
        for i, bin in enumerate(bins):
            hist[i] = np.sum((lat >= bin-avg/2) & (lat <= bin+avg/2))
        hist = npix2res(hist, np.sin(bins+avg/2) - np.sin(bins-avg/2))
        label = f'{graph.__class__.__name__} ({graph.N} pixels, {npix2res(graph.N):.0f} km, {np.sqrt(180*360/graph.N):.2f}°)'
        ax.plot(bins/np.pi*180, hist, '.', label=label)
        ax.axhline(npix2res(graph.N), linestyle='--', color='grey', zorder=3)

    ax.legend()
    ax.set_xlabel('latitude [°]')
    ax.set_ylabel('mean resolution [km]')

**Target 1**: 5° resolution on average ([WeatherBench](https://github.com/pangeo-data/WeatherBench) is 5.625°).

In [ ]:
npix = (params2npix('healpix', 16) + params2npix('icosahedral', 16)) / 2
print(f'target: {npix:.0f} pixels')
print(npix2params('cubed', npix))
print(npix2params('gaussian', npix))
print(npix2params('equiangular', npix))
print(npix2params('equiangular', npix*1.5))

plot_resolutions([
    pg.graphs.SphereHealpix(16),
    pg.graphs.SphereIcosahedral(16),
    pg.graphs.SphereCubed(22),
    pg.graphs.SphereGaussLegendre(45, nlon='ecmwf-octahedral'),  # ECMWF uses even numbers of rings only
#    pg.graphs.SphereEquiangular(32, 64),  # WeatherBench
    pg.graphs.SphereEquiangular(38, 76),
    pg.graphs.SphereEquiangular(46, 92),
])

**Target 2**: 100 km resolution on average.

* But let's see how far we can go before the GPU memory is filled.
* For cosmology, we could train with a single GPU on HEALPix with 10 subdivisions, i.e., ~12M pixels or a resolution of ~6.4km on the Earth.
  But it was for classification, hence the NN had no decoder.
* The ECMWF's IFS HRES currently runs on a reduced (octahedral) Gaussian grid of resolution O1280, i.e., ~6M pixels or a resolution of ~8.8km on the Earth.
* ERA5 is on a reduced (linear) Gaussian grid of resolution N320 (as older IFS), which should correspond to a resolution of ~32km.

In [ ]:
print(npix2res(params2npix('healpix', 10)))
print(npix2res(params2npix('gaussian', 2*1280)))

In [ ]:
npix = (params2npix('healpix', 64) + params2npix('icosahedral', 64)) / 2
print(f'target: {npix:.0f} pixels')
print(npix2params('cubed', npix))
print(npix2params('gaussian', npix))
print(npix2params('equiangular', npix))
print(npix2params('equiangular', npix*1.5))

plot_resolutions([
    pg.graphs.SphereHealpix(64),
    pg.graphs.SphereIcosahedral(64),
    pg.graphs.SphereCubed(87),
    pg.graphs.SphereGaussLegendre(204, nlon='ecmwf-octahedral'),
    pg.graphs.SphereEquiangular(150, 300),
    pg.graphs.SphereEquiangular(184, 368),
])

## 3 Positions of pixels (cells) and vertices

The positions of the pixels (graph vertices) are given by a PyGSP `graph`:
1. The 3D positions of the graph vertices that support the data are stored in `graph.coords`. 
2. The longitude and latitude positions are stored as signals as `graph.signals['lon']` and `graph.signals['lat']`.
3. `graph.coords` is set to 3D coordinates with `graph.set_coordinates('sphere', dim=3)`, and 2D lat-lon coordinates with `graph.set_coordinates('sphere', dim=2)`.

In [ ]:
graph = pg.graphs.SphereEquiangular(2, 4)

#graph.set_coordinates('sphere', dim=3)
print(f'{graph.coords.shape[0]} cells embedded in {graph.coords.shape[1]} dimensions')
print(graph.coords)

graph.set_coordinates('sphere', dim=2)
print(f'{graph.coords.shape[0]} cells embedded in {graph.coords.shape[1]} dimensions')
print(graph.coords)

A general definition of a pixel is as the set of points which are closest to a center.
Samplings can however define pixels differently, as HEALPix.

Assuming the graph vertices are at the center of cells supporting the data, those cells are given by the [Voronoi diagram](https://en.wikipedia.org/wiki/Voronoi_diagram) (the dual of a [Delaunay triangulation](https://en.wikipedia.org/wiki/Delaunay_triangulation)).
SciPy can compute a Voronoi diagram and give the positions of the Voronoi vertices at which the Voronoi cells intersect.

In [ ]:
graph.set_coordinates('sphere', dim=3)
graph = pg.graphs.SphereEquiangular(2, 4)
sv = SphericalVoronoi(graph.coords, radius=1, center=[0, 0, 0])
print(f'{sv.vertices.shape[0]} vertices embedded in {sv.vertices.shape[1]} dimensions')
print(sv.vertices)

In [ ]:
# HEALPix pixels aren't Voronoi cells.
graph = pg.graphs.SphereHealpix(1, k=8)
npix = graph.n_vertices
nside = np.sqrt(npix/12)
vertices = hp.boundaries(nside, range(npix), nest=graph.nest)
assert vertices.shape == (npix, 3, 4)

While the HEALPix pixels aren't Voronoi pixels, it's Voronoi pixels are almost equiarea.

In [ ]:
graphs = [
    pg.graphs.SphereHealpix(16),
    pg.graphs.SphereIcosahedral(16),
    pg.graphs.SphereCubed(22),
    pg.graphs.SphereGaussLegendre(45, nlon='ecmwf-octahedral'),
    pg.graphs.SphereEquiangular(38, 76),
    pg.graphs.SphereRandom(2817),
]

fig, axes = plt.subplots(1, len(graphs), figsize=(3*len(graphs), 3))
for graph, ax in zip(graphs, axes):
    sv = SphericalVoronoi(graph.coords, radius=1, center=[0, 0, 0])
    areas = sv.calculate_areas()
    np.testing.assert_allclose(areas.sum(), 4*np.pi)
    ax.hist(areas, bins=100)
    ax.set_title(graph.__class__.__name__)
fig.tight_layout()

In [ ]:
graphs = [
    pg.graphs.SphereHealpix(16),
    pg.graphs.SphereIcosahedral(16),
    pg.graphs.SphereCubed(22),
    pg.graphs.SphereGaussLegendre(45, nlon='ecmwf-octahedral'),
    pg.graphs.SphereEquiangular(38, 76),
    pg.graphs.SphereRandom(2817),
]
fig, axes = plt.subplots(1, len(graphs), figsize=(3*len(graphs), 3))
for graph, ax in zip(graphs, axes):
    G = pg.graphs.NNGraph(graph.coords, k=4, kernel=lambda d: d, kernel_width=1)
    ax.hist(G.W.data, bins=100)
    ax.set_title(graph.__class__.__name__)
fig.tight_layout()

## 4 Plotting

Code from <https://scipy.github.io/devdocs/generated/scipy.spatial.SphericalVoronoi.html>.

In [ ]:
def plot(graph, sv, edges=True, sphere=True, triangles=False, regions=True, ax=None):
    
    if ax is None:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

    if sphere:
        # plot the unit sphere for reference
        u = np.linspace(0, 2*np.pi, 100)
        v = np.linspace(0, np.pi, 100)
        x = np.outer(np.cos(u), np.sin(v))
        y = np.outer(np.sin(u), np.sin(v))
        z = np.outer(np.ones_like(u), np.cos(v))
        ax.plot_surface(x, y, z, color='y', alpha=0.1)

    # plot generator points (graph)
    graph.plot('b', 30, edges=edges, ax=ax, title='')

    # plot Voronoi vertices
    ax.scatter(*sv.vertices.T, c='g')

    # plot Delaunay triangles (as Euclidean polygons)
    # TODO: triangles' vertices are not sorted
    if triangles:
        t_vals = np.linspace(0, 1, 10)
        for region in sv._simplices:
            n = len(region)
            for i in range(n):
                start = sv.points[region][i]
                end = sv.points[region][(i + 1) % n]
                result = geometric_slerp(start, end, t_vals)
                ax.plot(result[..., 0],
                        result[..., 1],
                        result[..., 2],
                        c='k')

    # indicate Voronoi regions (as Euclidean polygons)
    if regions:
        sv.sort_vertices_of_regions()
        t_vals = np.linspace(0, 1, 10)
        for region in sv.regions:
            n = len(region)
            for i in range(n):
                start = sv.vertices[region][i]
                end = sv.vertices[region][(i + 1) % n]
                result = geometric_slerp(start, end, t_vals)
                # Returns a list when two vertices are at the same position.
                # Happens at the poles.
                result = np.asanyarray(result)
                ax.plot(result[..., 0],
                        result[..., 1],
                        result[..., 2],
                        c='k')

graphs = [
    pg.graphs.SphereHealpix(1, k=8),
    pg.graphs.SphereIcosahedral(1),
    pg.graphs.SphereCubed(2),
    pg.graphs.SphereGaussLegendre(4, 8),
    pg.graphs.SphereEquiangular(4, 8),
    pg.graphs.SphereRandom(20),
]

fig = plt.figure(figsize=(4*len(graphs), 4))
for i, graph in enumerate(graphs):
    ax = fig.add_subplot(1, len(graphs), i+1, projection='3d')
    sv = SphericalVoronoi(graph.coords, radius=1, center=[0, 0, 0])
    plot(graph, sv, edges=False, sphere=True, regions=True, ax=ax)
    ax.axis('off')
    title = graph.__class__.__name__
    title += f'\n{graph.n_vertices} pixels (graph vertices)'  # regions / points
    title += f'\n{sv.vertices.shape[0]} vertices (Delaunay triangles)'  # (region connections)'
    assert sv._simplices.shape[0] == sv.vertices.shape[0]
    ax.set_title(title)

## 5 Check spectral (hence equivariance) properties

In [ ]:
def plot_spectrum(graphs, n_eigenvalues=49, normalize=False, ax=None):
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 4))#figsize=(12, 8))

    for graph in graphs:
        graph.compute_fourier_basis(min(graph.N, n_eigenvalues))
        e = graph.e / graph.e[-1] if normalize else graph.e
        ax.plot(e, '.', label=f'{graph.__repr__(limit=2)}')
    ax.legend()#loc='upper left')

    eigenspace = 1
    vline = 1
    while vline <= min(n_eigenvalues, max(graph.N for graph in graphs)):
        ax.axvline(vline-0.5, linestyle='--', color='grey')
        eigenspace += 2
        vline += eigenspace

Comparison:
* HEALPix is best.
* Icosahedral and Cubed are not too bad.
    * Can be made better by using `kernel_width` from HEALPix.
* Equiangular is really bad. Worse than random. Seems to have the eigenstructure of a grid.
    * Can be made better with an `NNGraph`.
* They are all improved with more neighbors.

TODO:
* NNGraph: set sigma to mean(farthest neighbor) / 2
    * Same as for radius graph.
    * Mean pushes the width too far => most vertices are farther (as if uniform k ~ area ~ d²).
    * Kernel value 0.5 in the middle of the ball.
    * I remember it should have been in a paper. Shi-Malik?

In [ ]:
k = 20
width = pg.graphs.nngraphs.spherehealpix._OPTIMAL_KERNEL_WIDTHS[k][16]
fig, axes = plt.subplots(1, 2, figsize=(24, 5))
plot_spectrum([
    pg.graphs.SphereHealpix(16, k=k),
    pg.graphs.SphereIcosahedral(16, k=k, kernel_width=width),
    pg.graphs.SphereCubed(22, k=k, kernel_width=width),
    pg.graphs.SphereGaussLegendre(45, nlon='ecmwf-octahedral', k=k, kernel_width=width),  # better for k=8, same for k=20
#    pg.graphs.NNGraph(pg.graphs.SphereEquiangular(38, 76).coords, k=k, kernel_width=width),
#    pg.graphs.SphereRandom(2817, k=k, kernel_width=width),
], 100, ax=axes[0])
plot_spectrum([
    pg.graphs.SphereGaussLegendre(45, nlon='ecmwf-octahedral', k=k),  # better for k=40,60, same for k=20
#    pg.graphs.SphereEquiangular(38, 76),
    pg.graphs.NNGraph(pg.graphs.SphereEquiangular(38, 76).coords, k=k),
    pg.graphs.SphereRandom(2817, k=k),
], 100, ax=axes[1])

HEALPix:
* eigenspaces degrade from well-separated to continuous
* separation is better with more neighbors
* the more pixels, the farther the eigenspaces are good

In [ ]:
#nsides = [2, 4]
nsides = [8, 16]
fig, axes = plt.subplots(1, 2, figsize=(16, 4))
plot_spectrum([
#    pg.graphs.SphereHealpix(nsides[0], k=4, kernel_width=1),  # Faces are all quadrilaterals, but they are not equidistant. Voronoi pixels are different.
    pg.graphs.SphereHealpix(nsides[0], k=8),
    pg.graphs.SphereHealpix(nsides[0], k=20),
    pg.graphs.SphereHealpix(nsides[0], k=40),
    pg.graphs.SphereHealpix(nsides[0], k=60),
], 200, ax=axes[0], normalize=True)
plot_spectrum([
#    pg.graphs.SphereHealpix(nsides[1], k=4, kernel_width=1),
    pg.graphs.SphereHealpix(nsides[1], k=8),
    pg.graphs.SphereHealpix(nsides[1], k=20),
    pg.graphs.SphereHealpix(nsides[1], k=40),
    pg.graphs.SphereHealpix(nsides[1], k=60),
], 200, ax=axes[1], normalize=True)

In [ ]:
# k=3 is much better because there is only 1 distance, as all faces are triangles.
fig, axes = plt.subplots(1, 2, figsize=(16, 4))
plot_spectrum([
    pg.graphs.SphereIcosahedral(8, dual=False, k=5),
    pg.graphs.SphereIcosahedral(8, dual=False, k=6),  # Faces are mostly hexagons.
    pg.graphs.SphereIcosahedral(8, dual=False, k=7),
], 100, ax=axes[0])
plot_spectrum([
    pg.graphs.SphereIcosahedral(8, dual=True, k=3),  # Faces are all triangles.
    pg.graphs.SphereIcosahedral(8, dual=True, k=4),
    pg.graphs.SphereIcosahedral(8, dual=True, k=8),
], 100, ax=axes[1])

In [ ]:
plot_spectrum([
    pg.graphs.SphereIcosahedral(1, dual=True, k=19),  # Fully connected.
    pg.graphs.SphereIcosahedral(1, dual=True, k=3),  # Triangular faces.
    pg.graphs.SphereIcosahedral(1, dual=False, k=11),  # Fully connected.
    pg.graphs.SphereIcosahedral(1, dual=False, k=6),  # Hexagonal faces.
])

In [ ]:
# SphereCubed: equiangular is better.
# Faces are quadrilaterals, but k=4 doesn't help. Aren't they equidistant?
fig, axes = plt.subplots(1, 2, figsize=(16, 4))
plot_spectrum([
    pg.graphs.SphereCubed(22, 'equidistant'),
    pg.graphs.SphereCubed(22, 'equiangular'),
], 100, ax=axes[0])
plot_spectrum([
    pg.graphs.SphereCubed(22, 'equidistant', k=4),
    pg.graphs.SphereCubed(22, 'equiangular', k=4),
], 100, ax=axes[1])

In [ ]:
# SphereGaussLegendre: more neighbors and reduced (more uniform) helps.
fig, axes = plt.subplots(1, 2, figsize=(16, 4))
plot_spectrum([
    pg.graphs.SphereGaussLegendre(45, nlon='ecmwf-octahedral'),
    pg.graphs.SphereGaussLegendre(45, nlon=90),
], 100, ax=axes[0])
plot_spectrum([
    pg.graphs.SphereGaussLegendre(45, nlon='ecmwf-octahedral', k=40),
    pg.graphs.SphereGaussLegendre(45, nlon=90, k=40),
], 100, ax=axes[1])

In [ ]:
# SphereEquiangular: not better with more edges.
# Changing kernels doesn't help either.
# More neighbors do help.
G1 = pg.graphs.SphereEquiangular(38, 76)
plot_spectrum([
    G1,
    pg.graphs.NNGraph(G1.coords, k=20),
    pg.graphs.NNGraph(G1.coords, k=40),
#    pg.graphs.NNGraph(G1.coords, k=16, kernel=lambda d: 1/d, kernel_width=.25),
#    pg.graphs.NNGraph(G1.coords, k=4, kernel='gaussian', kernel_width=1),
#    pg.graphs.NNGraph(G1.coords, k=G1.N-1, kernel='gaussian', kernel_width=.5),
#    pg.graphs.NNGraph(G1.coords, kind='radius', radius=np.pi/20),
#    pg.graphs.NNGraph(G1.coords, kind='radius', radius=np.pi/10, kernel=lambda d: 1/d, kernel_width=1),
#    pg.graphs.NNGraph(G1.coords, k=4, kernel=lambda d: 1/d**2, kernel_width=1),
], 100)

In [ ]:
plot_spectrum([
    pg.graphs.NNGraph(G1.coords, k=20),
], 100)
plot_spectrum([
    pg.graphs.NNGraph(G1.coords, k=40),
], 100)

### Window function

[Tegmark, An icosahedron-based method for pixelizing the celestial sphere](https://arxiv.org/pdf/astro-ph/9610094.pdf)

In [ ]:
nside = 4
npix = 12*nside**2
w = 4*np.pi/npix * np.ones(npix)
wl = hp.anafast(w, lmax=9*nside)
plt.semilogy(wl)

In [ ]:
wl = hp.pixwin(nside, lmax=3*nside-1)
plt.plot(wl)

In [ ]:
nside = 4
npix = 12*nside**2
l, m = 1, 1
graph = pg.graphs.SphereHealpix(nside)
lat, lon = hp.pix2ang(nside, range(npix))
ylm = scipy.special.sph_harm(l, m, lon, lat)

ylm @ w

### Setting the edge weights

* Difference should be scaled by $1/d$, to get variation-per-unit-distance
* But longer edges should count less.
* Integration by summing edges connected to a vertex.
    * The more edges the more exact (quadrature).
    * Constant quadrature weights if edges go in uniform directions.

In [ ]:
x = np.linspace(0, 3)
y = np.exp(-x**2)

# Taylor series.
y1 = 1 / (1 + x**2)
y2 = 1 / (1 + x**2 + x**4/2)
y3 = 1 / (1 + x**2 + x**4/2 + x**6/6)

plt.plot(x, y)
plt.plot(x, y1)
plt.plot(x, y2)
plt.plot(x, y3)

In [ ]:
graph = pg.graphs.SphereHealpix(4)
plot_spectrum([
    graph,
    
    # Not so sensible to the kernel width.
    pg.graphs.SphereHealpix(4, kernel_width=0.9*graph.kernel_width),  # still ok
#    pg.graphs.SphereHealpix(4, kernel_width=0.6*graph.kernel_width),  # very bad

    # 1/d is not the solution.
    #pg.graphs.NNGraph(graph.coords, kernel=lambda d: 1/d, kernel_width=graph.kernel_width),

    # Taylor series.
    pg.graphs.NNGraph(graph.coords, kernel=lambda d: 1/(1+d**2), kernel_width=graph.kernel_width),
    #pg.graphs.NNGraph(graph.coords, kernel=lambda d: 1/(1+d**2+d**4/2), kernel_width=graph.kernel_width),
    pg.graphs.NNGraph(graph.coords, kernel=lambda d: 1/(1+d**2+d**4/2+d**6/6), kernel_width=graph.kernel_width),
], 200)

In [ ]:
_OPTIMAL_KERNEL_WIDTHS = pg.graphs.nngraphs.spherehealpix._OPTIMAL_KERNEL_WIDTHS

x = np.array(list(_OPTIMAL_KERNEL_WIDTHS[8].keys()))
x = 12*x**2  # nside to npix
plt.loglog(x, list(_OPTIMAL_KERNEL_WIDTHS[8].values()))
plt.loglog(x, list(_OPTIMAL_KERNEL_WIDTHS[20].values()))
plt.loglog(x, list(_OPTIMAL_KERNEL_WIDTHS[40].values()))
plt.loglog(x, list(_OPTIMAL_KERNEL_WIDTHS[60].values()))

# width = cst / subdivisions = cst / sqrt(npix)
# width = cst * distance = cst * sqrt(area)
# weights = kernel(distances/width)

In [ ]:
graph = pg.graphs.SphereHealpix(8, kernel=lambda d: d, kernel_width=1, k=4)
#min = np.min(graph.W.toarray(), axis=0)
d = np.max(graph.W.toarray(), axis=1)
#d = np.mean(graph.W.toarray(), axis=1)
#d = np.median(graph.W.toarray(), axis=1)

plt.hist(d, bins=100);
#plt.hist(graph.W.data, bins=100);

In [ ]:
neighbors = [8, 20, 40, 60]
#neighbors = np.arange(10, 200, 5)
radius_mean = []
radius_median = []
radius_max = []
for k in neighbors:
    graph = pg.graphs.SphereHealpix(8, kernel=lambda d: d, kernel_width=1, k=k)
    radius_mean.append(np.mean(graph.W.data))
    radius_median.append(np.median(graph.W.data))
    radius_max.append(np.max(graph.W.data))
# All statistics have the same asymptotic behaviour.
plt.plot(neighbors, radius_mean/radius_mean[-1], '.-', label='mean')
plt.plot(neighbors, radius_median/radius_median[-1], '.-', label='median')
plt.plot(neighbors, radius_max/radius_max[-1], '.-', label='max')

for nside in [32, 64, 128, 256, 512, 1024]:
    y = np.array([_OPTIMAL_KERNEL_WIDTHS[k][nside] for k in neighbors])
    y /= y[-1]
    plt.plot(neighbors, y, '.-', label=f'nside={nside}')

#x = np.array(neighbors)
#x = np.linspace(8, 60, 100)
#y = np.linspace(y[0], 1, 100)
#plt.plot(x, y, '--', label='linear', c=(0.8,)*3)

plt.legend()

In [ ]:
def nside2pixradius(nside):
    nside = 8
    npix = 12*nside**2
    pix_area = 4*np.pi / npix
    pix_radius = np.sqrt(pix_area)
    return pix_radius

nside = 8
r = 4 * nside2pixradius(nside)
graph = pg.graphs.SphereHealpix(nside, kind='radius', radius=r)
plt.hist(graph.d, bins=100);

* On a quasi-uniform sampling, a kNN graph is quasi a radius graph, with the radius given by the farthest connected pair of vertices.
* `radius` grows as `sqrt(neighbors)`, and `area=radius**2` as `neighbors`.

In [ ]:
nside = 8
radiuses = np.linspace(1, 8, 20) * nside2pixradius(nside)

radius_mean = []
radius_median = []
radius_max = []
neighbors = []
neighbors_std = []
for r in radiuses:
    graph = pg.graphs.SphereHealpix(nside, kernel=lambda d: d, kernel_width=1, kind='radius', radius=r)
    neighbors.append(np.mean(graph.d))
    neighbors_std.append(np.std(graph.d))
    
    radius_mean.append(np.mean(graph.W.data))
    radius_median.append(np.median(graph.W.data))
    radius_max.append(np.max(graph.W.data))
#plt.plot(neighbors, radius_mean, '.-', label='mean')
#plt.plot(neighbors, radius_median, '.-', label='median')
#plt.plot(neighbors, radius_max, '.-', label='max')
plt.plot(neighbors, radius_mean/radius_mean[-1], '.-', label='mean')
plt.plot(neighbors, radius_median/radius_median[-1], '.-', label='median')
plt.plot(neighbors, radius_max/radius_max[-1], '.-', label='max')
area = np.array(radius_mean)**2
plt.plot(neighbors, area/area[-1], '.-', label='area')

#plt.plot(neighbors, radius_max/radius_max[-1], '.-', label='max')
#plt.plot(radiuses, neighbors, '.-')
plt.plot(neighbors, radiuses, '.-', label='radius')

for nside in [32, 64, 128, 256, 512, 1024]:
    neighbors = [8, 20, 40, 60]
    y = np.array([_OPTIMAL_KERNEL_WIDTHS[k][nside] for k in neighbors])
    y /= y[-1] / 0.6
    plt.plot(neighbors, y, '.-', label=f'nside={nside}')

plt.legend()

In [ ]:
# The distribution of #neighbors is well concentrated.
#plt.plot(radiuses, neighbors, '.-')
plt.plot(radiuses, neighbors_std, '.-')

In [ ]:
k = 40
nside = 8
npix = 12*nside**2
G1 = pg.graphs.SphereHealpix(16, k=k)

# Makes it better.
G2 = pg.graphs.SphereIcosahedral(8, k=k)
G3 = pg.graphs.SphereIcosahedral(8, k=k, kernel_width=G1.kernel_width)
G2 = pg.graphs.SphereIcosahedral(8, dual=True, k=k)
G3 = pg.graphs.SphereIcosahedral(8, dual=True, k=k, kernel_width=G1.kernel_width)
G2 = pg.graphs.SphereCubed(11, k=k)
G3 = pg.graphs.SphereCubed(11, k=k, kernel_width=G1.kernel_width)
G2 = pg.graphs.SphereGaussLegendre(45, 'ecmwf-octahedral', k=k)
G3 = pg.graphs.SphereGaussLegendre(45, 'ecmwf-octahedral', k=k, kernel_width=G1.kernel_width)

# Makes it worse.
#G2 = pg.graphs.SphereGaussLegendre(20, k=k)
#G3 = pg.graphs.SphereGaussLegendre(20, k=k, kernel_width=G1.kernel_width)
#G2 = pg.graphs.NNGraph(pg.graphs.SphereEquiangular(20).coords, k=k)
#G3 = pg.graphs.NNGraph(pg.graphs.SphereEquiangular(20).coords, k=k, kernel_width=G1.kernel_width)

#G4 = pg.graphs.SphereIcosahedral(8, k=6)
print(G1)
print(G2)
print(G3)
#print(G4)

plot_spectrum([
    G1,
#    G2,
    G3,
#    G4,
], 100)

### Vertex weights as areas

Can be better or worse.

In [ ]:
# Makes it a bit better.
graph = pg.graphs.SphereEquiangular(10)
# Makes it worse.
graph = pg.graphs.NNGraph(pg.graphs.SphereEquiangular(10).coords, k=10)
graph = pg.graphs.SphereGaussLegendre(20, k=20)
# Not much change (quasi-equiarea).
graph = pg.graphs.SphereIcosahedral(8)
graph = pg.graphs.SphereHealpix(8)
graph = pg.graphs.SphereCubed(8, k=20)

#plot_spectrum([graph])

sv = SphericalVoronoi(graph.coords)
areas = sv.calculate_areas()
plt.plot(areas, '.')

I = np.identity(len(areas))
D = np.diag(areas)
Di = np.diag(1/areas)

eI, UI = scipy.linalg.eigh(graph.L.toarray(), I)
eD, UD = scipy.linalg.eigh(graph.L.toarray(), D)
eDi, UDi = scipy.linalg.eigh(graph.L.toarray(), Di)

n = 100
plt.figure(figsize=(18, 4))
plt.plot(eI[:n], '.')
plt.plot(eD[:n]*np.mean(areas), '.')
#plt.plot(eDi[:n]/np.mean(areas), '.')

### Mesh Laplacian

In [ ]:
import trimesh

In [ ]:
graph = pg.graphs.SphereHealpix(2)
sv = SphericalVoronoi(graph.coords)

In [ ]:
trimesh.Trimesh(sv.vertices)